In [21]:
import os
from nltk.corpus import stopwords
import re 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from spacy.lang.es import 
import warnings

warnings.filterwarnings('ignore')

def remove_stopword(x, lista_stopwords):
    return [y for y in x if y not in lista_stopwords]

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers. Also, we added the unicode line for accent marks'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text) #Punctuations...
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    #text = unidecode.unidecode(text)
    return text

def make_clean_dataframe(stopwords_espaniol, path_textos):
    #Accedo al path y jalo toda la info
    dicc={}
    for nombre_doc in os.listdir(path_textos):
        text_string = open(path_textos+'/'+nombre_doc).read()
        dicc[nombre_doc[:-4]] = text_string
    
    #Limpio y transformo el texto.
    dataframe = pd.DataFrame(dicc,index=[0]).T.rename(columns={0:'texto'}).reset_index()
    dataframe['temp_list'] = dataframe['texto'].apply(lambda x: clean_text(x))
    dataframe['temp_list'] = dataframe['temp_list'].apply(lambda x: str(x).split())
    dataframe['texto_limpio'] = dataframe['temp_list'].apply(lambda x: remove_stopword(x, stopwords_espaniol))
    dataframe = dataframe.rename(columns={'index':'nombre_doc'})

    for k,v in dataframe['texto_limpio'].items():
        dataframe.loc[k,'raw_clean_text'] = ' '.join(dataframe.loc[k,'texto_limpio'])
    
    return dataframe

agg_stopword = ['s', '2018','31','diciembre','financieros','000','2019','nota','grupo','valor','2017','resultados','compania','1',
 'total','consolidados','consolidado','razonable','gerencia','ciento','c','activos','cuentas','neto','us','efectivo','fecha','peru',
 'inretail','2','3','importe', 'aproximadamente','b','respectivamente','ver','ano','si','vida','anos','4','d','5','i','www','com',
 'aa', 'aaa', 'aaahipotecario', 'aaatat', 'aamnto', 'ab','ir','email','mes','niif','fmiv','bbb','ok','mzo','inc','alicorp','notas','dic','a','y']


stopwords_espaniol = list(STOP_WORDS)
stopwords_espaniol.extend(agg_stopword)

In [22]:
def jaccard_score(str1, str2):
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c) / (len(a) + len(b) - len(c)))

In [23]:
dataframe = make_clean_dataframe(stopwords_espaniol, 'data')
dataframe

,nombre_doc,texto,temp_list,texto_limpio,raw_clean_text
0,NOTAS_ALICORP_2018_1Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[marzo, expresados, miles, soles, principios, ...",marzo expresados miles soles principios practi...
1,NOTAS_ALICORP_2018_2Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[junio, expresados, miles, soles, principios, ...",junio expresados miles soles principios practi...
2,NOTAS_ALICORP_2018_3Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[setiembre, expresados, miles, soles, principi...",setiembre expresados miles soles principios pr...
3,NOTAS_ALICORP_2018_4Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[expresados, miles, soles, principios, practic...",expresados miles soles principios practicas co...
4,NOTAS_ALICORP_2019_1Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[marzo, expresados, miles, soles, principios, ...",marzo expresados miles soles principios practi...
5,NOTAS_ALICORP_2019_2Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[separados, junio, expresados, miles, soles, p...",separados junio expresados miles soles princip...
6,NOTAS_ALICORP_2019_3Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[separados, septiembre, expresados, miles, sol...",separados septiembre expresados miles soles pr...
7,NOTAS_ALICORP_2019_4Q,alicorp s a a y subsidiarias notas a los estad...,"[alicorp, s, a, a, y, subsidiarias, notas, a, ...","[subsidiarias, expresados, miles, soles, princ...",subsidiarias expresados miles soles principios...
8,NOTAS_ALICORP_2020_1Q,alicorp s a a y subsidiarias notas a los estad...,"[alicorp, s, a, a, y, subsidiarias, notas, a, ...","[subsidiarias, marzo, expresados, miles, soles...",subsidiarias marzo expresados miles soles acti...


In [56]:
dicc_replace = {'coberturas':'cobertura','arrendamientos':'arrendamiento','bonos':'bono','opciones':'opcion','pasivos':'pasivo','corporativos':'corporativo'}
dataframe['raw_clean_text'].replace(dicc_replace, inplace=True)

In [57]:
dataframe['Jaccard'] = np.nan
dataframe['Jaccard cr 4Q 2019'] = np.nan
for i in dataframe.index:
    try:
        dataframe['Jaccard'][i] = jaccard_score(dataframe.loc[i,'raw_clean_text'], dataframe.loc[i + 1, 'raw_clean_text'])
        dataframe['Jaccard cr 4Q 2019'][i] = jaccard_score(dataframe.loc[7,'raw_clean_text'], dataframe.loc[i+1, 'raw_clean_text'])
    except:
        pass
dataframe

,nombre_doc,texto,temp_list,texto_limpio,raw_clean_text,Jaccard,Jaccard cr 4Q 2019
0,NOTAS_ALICORP_2018_1Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[marzo, expresados, miles, soles, principios, ...",marzo expresados miles soles principios practi...,0.643917,0.260692
1,NOTAS_ALICORP_2018_2Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[junio, expresados, miles, soles, principios, ...",junio expresados miles soles principios practi...,0.830357,0.261477
2,NOTAS_ALICORP_2018_3Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[setiembre, expresados, miles, soles, principi...",setiembre expresados miles soles principios pr...,0.592965,0.226923
3,NOTAS_ALICORP_2018_4Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[expresados, miles, soles, principios, practic...",expresados miles soles principios practicas co...,0.384091,0.277895
4,NOTAS_ALICORP_2019_1Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[marzo, expresados, miles, soles, principios, ...",marzo expresados miles soles principios practi...,0.825949,0.303226
5,NOTAS_ALICORP_2019_2Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[separados, junio, expresados, miles, soles, p...",separados junio expresados miles soles princip...,0.573134,0.246085
6,NOTAS_ALICORP_2019_3Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[separados, septiembre, expresados, miles, sol...",separados septiembre expresados miles soles pr...,0.246085,1.000000
7,NOTAS_ALICORP_2019_4Q,alicorp s a a y subsidiarias notas a los estad...,"[alicorp, s, a, a, y, subsidiarias, notas, a, ...","[subsidiarias, expresados, miles, soles, princ...",subsidiarias expresados miles soles principios...,0.172000,0.172000
8,NOTAS_ALICORP_2020_1Q,alicorp s a a y subsidiarias notas a los estad...,"[alicorp, s, a, a, y, subsidiarias, notas, a, ...","[subsidiarias, marzo, expresados, miles, soles...",subsidiarias marzo expresados miles soles acti...,NaN,NaN


In [58]:
dataframe[['nombre_doc','Jaccard','Jaccard cr 4Q 2019']]

,nombre_doc,Jaccard,Jaccard cr 4Q 2019
0,NOTAS_ALICORP_2018_1Q,0.643917,0.260692
1,NOTAS_ALICORP_2018_2Q,0.830357,0.261477
2,NOTAS_ALICORP_2018_3Q,0.592965,0.226923
3,NOTAS_ALICORP_2018_4Q,0.384091,0.277895
4,NOTAS_ALICORP_2019_1Q,0.825949,0.303226
5,NOTAS_ALICORP_2019_2Q,0.573134,0.246085
6,NOTAS_ALICORP_2019_3Q,0.246085,1.000000
7,NOTAS_ALICORP_2019_4Q,0.172000,0.172000
8,NOTAS_ALICORP_2020_1Q,NaN,NaN


#### Hasta acá tenemos 9 estados financieros de una misma empresa ya limpios de caracteres y palabras basura. Ahora tenemos que Lemmatizar las palabras, es decir, volverlas a su raíz para mejor procesamiento. Luego, intentaremos clasificar los documentos según el modelo No Supervisado: Latent Dirichlet Allocation.

Recuerda en que la diferencia entre Lemmatizer y Stemmer yace en la metodología de la reducción de palabras. Mientras Lemmatizer se basa en un análisis morfológico de las palabras y requiere de un diccionario de especificación, Stemmer se basa en cortar prefijos y sufijos comunes a las palabras involucradas en el texto. 

In [59]:
from nltk.stem import PorterStemmer

def porter_stemmer(word):
    stemmer = PorterStemmer()
    return stemmer.stem(word)

In [60]:
dataframe['raw_clean_text'].str.split()

0    [marzo, expresados, miles, soles, principios, ...
1    [junio, expresados, miles, soles, principios, ...
2    [setiembre, expresados, miles, soles, principi...
3    [expresados, miles, soles, principios, practic...
4    [marzo, expresados, miles, soles, principios, ...
5    [separados, junio, expresados, miles, soles, p...
6    [separados, septiembre, expresados, miles, sol...
7    [subsidiarias, expresados, miles, soles, princ...
8    [subsidiarias, marzo, expresados, miles, soles...
Name: raw_clean_text, dtype: object

In [62]:
porter_stemmer('subsidiarias')

'subsidiaria'

In [63]:
dict_data = {k:v.split() for k, v in zip(dataframe['nombre_doc'],dataframe['raw_clean_text'])}

In [64]:
doc_clean = [*dataframe['raw_clean_text'].str.split()]

In [65]:
import gensim
from gensim import corpora
from gensim.models import CoherenceModel

#Debemos crear un diccionario de nuestro corpus (lista de docs), donde cada término único sea asignado a un index. 

#Convirtiendo la lista de documentos corpus en una Matriz de términos de documentos, usando el diccionario dict_data

dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [66]:
dictionary = corpora.Dictionary(doc_clean)

#### Latent Dirichlet Allocation model: 

In [37]:
LDA = gensim.models.ldamodel.LdaModel

ldamodel = LDA(doc_term_matrix, num_topics=5, id2word=dictionary, passes=40,random_state=300)

In [38]:
print(ldamodel.print_topics(num_topics=5, num_words=6))

[(0, '0.015*"arrendamiento" + 0.014*"inversiones" + 0.012*"activo" + 0.012*"arrendamientos" + 0.011*"pasivo" + 0.010*"subsidiaria"'), (1, '0.012*"instrumentos" + 0.011*"coberturas" + 0.011*"cobertura" + 0.011*"ingresos" + 0.010*"contables" + 0.010*"inversiones"'), (2, '0.001*"cobrar" + 0.001*"emision" + 0.001*"serie" + 0.001*"inversiones" + 0.001*"marzo" + 0.001*"contables"'), (3, '0.019*"arrendamiento" + 0.014*"bonos" + 0.014*"emision" + 0.010*"subsidiarias" + 0.009*"vigentes" + 0.009*"bolivianos"'), (4, '0.023*"cobrar" + 0.014*"marzo" + 0.008*"vitapro" + 0.008*"distribucion" + 0.008*"deterioro" + 0.006*"mar"')]


#### Hierarchical Dirichlet process:

En este caso, el modelo debería poner el número de tópicos que necesitas.

In [50]:
from gensim.models.hdpmodel import HdpModel

hdpmodel = HdpModel(corpus= doc_term_matrix, id2word=dictionary, random_state=300)

hdpmodel.show_topics()

[(0,
  '0.014*arrendamiento + 0.012*inversiones + 0.012*subsidiaria + 0.010*pasivo + 0.010*activo + 0.010*arrendamientos + 0.009*cambio + 0.008*derecho + 0.008*call + 0.007*acciones + 0.007*enero + 0.006*spread + 0.006*contables + 0.006*corporativos + 0.006*separados + 0.006*capital + 0.006*emision + 0.006*tipo + 0.006*realizo + 0.006*serie'),
 (1,
  '0.009*instrumentos + 0.009*inversiones + 0.008*contables + 0.008*ingresos + 0.008*cobertura + 0.008*coberturas + 0.007*serie + 0.007*soles + 0.007*emision + 0.007*costo + 0.007*subsidiaria + 0.006*forma + 0.006*informacion + 0.006*enero + 0.006*opciones + 0.006*vencimiento + 0.006*corrienteno + 0.005*bimbo + 0.005*periodo + 0.005*bonos'),
 (2,
  '0.009*dividendos + 0.008*inversiones + 0.008*serie + 0.008*emision + 0.007*contables + 0.007*arrendamiento + 0.007*informacion + 0.007*o + 0.006*arrendamientos + 0.006*instrumentos + 0.006*bonos + 0.006*opciones + 0.006*ecuador + 0.006*periodo + 0.006*pasivo + 0.006*activo + 0.006*plazo + 0.006*e

#### Métricas del modelo: 

Perplexity: Qué tan bueno es el modelo con respecto a palabras buenas.

Coherence Score: Qué tanto se sustenta el modelo con respecto a las palabras aledañas.

In [39]:
print('Perplexity: {}'.format(ldamodel.log_perplexity(doc_term_matrix))) #Mientras más bajo mejor.

#Cohere Score:
coherence_model_lda = CoherenceModel(model=ldamodel, texts= doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: {}'.format(coherence_lda))

Perplexity: -6.39911258259982
Coherence Score: 0.27063647144685754


In [40]:
ldamodel2 = LDA(doc_term_matrix, num_topics=3, id2word=dictionary, passes=60,random_state=300)
coherence_model_lda = CoherenceModel(model=ldamodel2, texts= doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: {}'.format(coherence_lda))

Coherence Score: 0.24235503368297784


In [41]:
#Ahora le pondremos 600 passes
ldamodel2 = LDA(doc_term_matrix, num_topics=7, id2word=dictionary, passes=600,random_state=300)
coherence_model_lda = CoherenceModel(model=ldamodel2, texts= doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: {}'.format(coherence_lda))

Coherence Score: 0.36707448323881103


In [42]:
print(ldamodel2.print_topics(num_topics=5, num_words=6))

[(5, '0.020*"arrendamiento" + 0.015*"bonos" + 0.015*"emision" + 0.011*"subsidiarias" + 0.009*"vigentes" + 0.009*"bolivianos"'), (4, '0.001*"construcciones" + 0.001*"edificios" + 0.001*"unidades" + 0.001*"obras" + 0.001*"maquinaria" + 0.001*"plantas"'), (0, '0.026*"cobrar" + 0.015*"marzo" + 0.009*"deterioro" + 0.009*"vitapro" + 0.009*"distribucion" + 0.009*"mar"'), (2, '0.001*"construcciones" + 0.001*"edificios" + 0.001*"unidades" + 0.001*"obras" + 0.001*"maquinaria" + 0.001*"plantas"'), (1, '0.013*"instrumentos" + 0.012*"coberturas" + 0.012*"cobertura" + 0.011*"ingresos" + 0.011*"contables" + 0.011*"inversiones"')]


In [67]:
ldamodel3 = LDA(corpus = doc_term_matrix, num_topics=7, id2word=dictionary, passes=600,random_state=300,alpha=0.05)
coherence_model_lda = CoherenceModel(model=ldamodel2, texts= doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: {}'.format(coherence_lda))

Coherence Score: 0.36707448323881103


In [68]:
print(ldamodel3.print_topics(num_topics=5, num_words=6))

[(5, '0.020*"arrendamiento" + 0.015*"bonos" + 0.015*"emision" + 0.011*"subsidiarias" + 0.009*"vigentes" + 0.009*"bolivianos"'), (4, '0.001*"extension" + 0.001*"estime" + 0.001*"fiscales" + 0.001*"fiscal" + 0.001*"factores" + 0.001*"incertidumbre"'), (0, '0.026*"cobrar" + 0.015*"marzo" + 0.009*"deterioro" + 0.009*"vitapro" + 0.009*"distribucion" + 0.009*"mar"'), (2, '0.001*"extension" + 0.001*"estime" + 0.001*"fiscales" + 0.001*"fiscal" + 0.001*"factores" + 0.001*"incertidumbre"'), (1, '0.013*"instrumentos" + 0.012*"cobertura" + 0.012*"coberturas" + 0.011*"ingresos" + 0.011*"contables" + 0.011*"inversiones"')]


Podemos ver que algunos de las palabras dentro de los tópicos nos pueden ser útiles. Pero, quedan las preguntas:

¿Cómo sé cuántos tópicos son óptimos dentro de mis datos? 

¿Cómo veo el puntaje de un tópico por documento?

#### Optimizando el Número de tópicos: 

Fuente: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [45]:
import numpy as np
import tqdm

def compute_coherence_values_lda(corpus, texts, id2word, alpha, n_topics):
    "Hace el modelo y luego valida los resultados"
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=n_topics, 
                                           random_state=133,
                                           passes=50,
                                           alpha=alpha)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

def compute_coherence_values_hdp(corpus, texts, id2word, alpha, beta, tau):
    return 0

grid = {}
grid['Validation_Set'] = {}
# Rango de tópicos
min_topics = 3
max_topics = 10
topics_range = range(min_topics, max_topics, 1)
# Rango de optimización del alpha
alpha = list(np.arange(0.01, 1, 0.25))
alpha.append('symmetric')
# Rango de optimización del beta
beta = list(np.arange(0.01, 1, 0.25))
beta.append('symmetric')
# VTest de validación
num_of_docs = len(doc_clean)

model_results = {
                 'Topics': [],
                 'Alpha': [],
                 'Coherence': []
                }
n=0
for n_topic in topics_range:
    n += 1
    print("Va {} grupo de modelos ejecutado para {} de tópicos".format(n,n_topic))
    i=0
    for a in alpha:
        i+=1
        coherence_value = compute_coherence_values_lda(corpus = doc_term_matrix, texts=doc_clean, n_topics=n_topic, alpha=a,id2word=dictionary)
        print('Va {} modelo entrenado con un score de {}'.format(i, coherence_value))
        model_results['Topics'].append(n_topic)
        model_results['Alpha'].append(a)
        model_results['Coherence'].append(coherence_value)

resultados = pd.DataFrame(model_results)

Va 1 grupo de modelos ejecutado para 3 de tópicos
Va 1 modelo entrenado con un score de 0.2404934014818019
Va 2 modelo entrenado con un score de 0.3842900891470514
Va 3 modelo entrenado con un score de 0.38122253938677675
Va 4 modelo entrenado con un score de 0.38122253938677675
Va 5 modelo entrenado con un score de 0.3696535061736827
Va 2 grupo de modelos ejecutado para 4 de tópicos
Va 1 modelo entrenado con un score de 0.26706563521896154
Va 2 modelo entrenado con un score de 0.2495850882560542
Va 3 modelo entrenado con un score de 0.2565842424763964
Va 4 modelo entrenado con un score de 0.2565842424763964
Va 5 modelo entrenado con un score de 0.2655812412564498
Va 3 grupo de modelos ejecutado para 5 de tópicos
Va 1 modelo entrenado con un score de 0.2945573726610323
Va 2 modelo entrenado con un score de 0.3711982774357735
Va 3 modelo entrenado con un score de 0.364854330401864
Va 4 modelo entrenado con un score de 0.3872173983424222
Va 5 modelo entrenado con un score de 0.3209261693

In [46]:
resultados.sort_values('Coherence',ascending=False).head()

,Topics,Alpha,Coherence
23,7,0.76,0.418813
13,5,0.76,0.387217
1,3,0.26,0.384290
2,3,0.51,0.381223
3,3,0.76,0.381223


In [69]:
ldamodel3 = LDA(corpus = doc_term_matrix, num_topics=5, id2word=dictionary, passes=600,random_state=300,alpha=0.75)
coherence_model_lda = CoherenceModel(model=ldamodel2, texts= doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: {}'.format(coherence_lda))

Coherence Score: 0.36707448323881103


In [70]:
print(ldamodel3.print_topics(num_topics=5, num_words=6))

[(5, '0.001*"iquitos" + 0.001*"distribuidor" + 0.001*"muebles" + 0.001*"consorcio" + 0.001*"maquinaria" + 0.001*"obras"'), (0, '0.013*"arrendamiento" + 0.012*"activo" + 0.012*"inversiones" + 0.011*"arrendamientos" + 0.010*"pasivo" + 0.010*"derecho"'), (4, '0.001*"iquitos" + 0.001*"distribuidor" + 0.001*"muebles" + 0.001*"consorcio" + 0.001*"maquinaria" + 0.001*"obras"'), (3, '0.019*"arrendamiento" + 0.015*"bonos" + 0.015*"emision" + 0.011*"subsidiarias" + 0.009*"vigentes" + 0.009*"bolivianos"'), (1, '0.012*"instrumentos" + 0.011*"inversiones" + 0.011*"ingresos" + 0.011*"contables" + 0.011*"coberturas" + 0.011*"cobertura"')]


In [71]:
print(ldamodel3.print_topics(num_topics=5, num_words=6))

[(2, '0.014*"swap" + 0.014*"unwind" + 0.014*"equivalente" + 0.014*"registro" + 0.014*"acciones" + 0.009*"perdida"'), (4, '0.001*"iquitos" + 0.001*"distribuidor" + 0.001*"muebles" + 0.001*"consorcio" + 0.001*"maquinaria" + 0.001*"obras"'), (1, '0.012*"instrumentos" + 0.011*"inversiones" + 0.011*"ingresos" + 0.011*"contables" + 0.011*"coberturas" + 0.011*"cobertura"'), (3, '0.019*"arrendamiento" + 0.015*"bonos" + 0.015*"emision" + 0.011*"subsidiarias" + 0.009*"vigentes" + 0.009*"bolivianos"'), (5, '0.001*"iquitos" + 0.001*"distribuidor" + 0.001*"muebles" + 0.001*"consorcio" + 0.001*"maquinaria" + 0.001*"obras"')]


### PyLDAvis

In [73]:
import pyLDAvis as ldavis
import pyLDAvis.gensim

ldavis.enable_notebook()
prepared_data = ldavis.gensim.prepare(ldamodel3, doc_term_matrix, dictionary)

prepared_data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.174553 -0.056331       1        1  42.416569
0     -0.052591 -0.000446       2        1  42.098217
3     -0.001401  0.144304       3        1  13.153480
2      0.146708 -0.057401       4        1   2.258614
5      0.040919 -0.015063       5        1   0.036563
4      0.040919 -0.015063       6        1   0.036563, topic_info=    Category       Freq            Term      Total  loglift  logprob
447  Default  33.000000   arrendamiento  33.000000  30.0000  30.0000
373  Default  11.000000        acciones  11.000000  29.0000  29.0000
38   Default  19.000000      coberturas  19.000000  28.0000  28.0000
37   Default  19.000000       cobertura  19.000000  27.0000  27.0000
377  Default  22.000000  arrendamientos  22.000000  26.0000  26.0000
..       ...        ...             ...        ...      ...      ...
185   Topic6   0.001799          precio   4.081047   0.1871  -6.7238
138   Topic6   0.001799             mar   6.982192  -0.3499  -6.7238
137   Topic6   0.001799      maquinaria   2.360279   0.7347  -6.7238
224   Topic6   0.001799   significativo   7.909870  -0.4747  -6.7238
41    Topic6   0.001799         computo   3.285286   0.4040  -6.7238

[476 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
253       1  0.275355     abril
253       2  0.137678     abril
253       3  0.550710     abril
373       2  0.841259  acciones
373       4  0.084126  acciones
...     ...       ...       ...
252       3  0.305205  vigentes
371       1  0.226489   vitapro
371       2  0.792710   vitapro
441       4  0.759415   volumen
532       2  0.913409    volver

[477 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3, 6, 5])

TypeError: '>' not supported between instances of 'operator.itemgetter' and 'list'

### Fuentes a considerar:

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

https://github.com/mattilyra/pydataberlin-2017/blob/master/notebook/EvaluatingUnsupervisedModels.ipynb

https://www.youtube.com/watch?v=T05t-SqKArY

https://www.youtube.com/watch?v=ZkAFJwi-G98

https://radimrehurek.com/gensim/models/ldamodel.html

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

https://towardsdatascience.com/perplexity-intuition-and-derivation-105dd481c8f3

https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

https://www.aclweb.org/anthology/D12-1087/

https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

To-Supervised:

https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28